In [1]:
import numpy as np 
import os
import cv2

In [2]:
dir_path = "Dataset/" 
actions = np.array(os.listdir(dir_path))

## formatting the data

In [3]:
#Pre processing data and creating labels 
from tensorflow.keras.utils import to_categorical

label_map = {label: num for num, label in enumerate(actions)}

In [20]:
print(label_map)

{'Blue': 0, 'Family': 1, 'Happy': 2, 'Man': 3}


In [4]:
labels = []

for action in actions: 
    file_list = os.listdir(dir_path+"/"+action)

    for video in range(len(file_list)): 
        labels.append(label_map[action])


In [6]:
import pandas as pd
# from keras.applications.vgg16 import VGG16

# vgg16_model = VGG16(weights ='imagenet', include_top=False, input_shape=(224, 224, 3))
test_actions = ['Blue','Happy','Man', 'Family']

video_features = []
lengths = []
for action in test_actions: 
    file_list = os.listdir(dir_path+'/'+action)
    num_files = len(file_list)
       
    for file in range(num_files):
        file_name = file_list[file]
        cap = cv2.VideoCapture(dir_path+'/'+action+'/'+file_name)

        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame,(224,224))
            frames.append(frame)
            lengths.append(len(frame))
        cap.release()
        features = np.array(frames)
        # features = np.mean(features, axis=(1,2))

        video_features.append(features)

video_features = np.array(video_features)
df = pd.DataFrame({'class': labels, 'features': video_features})
df = df.sample(frac=1).reset_index(drop=True)

c:\Users\amyan\anaconda3\envs\IPCV\lib\site-packages\ipykernel_launcher.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [5]:
def extract_frames_and_resize(video_path, frame_size=(244,244)): 
    frames = []
    lengths = []
    cap = cv2.VideoCapture(video_path)
    while True: 
        ret,frame = cap.read()
        if not ret: 
            break 
        frame = cv2.resize(frame, frame_size)
        frames.append(frame)
        lengths.append(len(frame))
 
    cap.release()
    return frames, lengths


In [6]:
data = []
for action in actions: 
    file_list = os.listdir(dir_path+'/'+action)
    num_files = len(file_list)
       
    for file in file_list:
        file_path = os.path.join(dir_path, action, file)
        frames, lengths = extract_frames_and_resize(file_path)

        data.append(frames)
    

In [7]:
data = np.array(data)
labels = np.array(labels)

c:\Users\amyan\anaconda3\envs\IPCV\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [8]:
df.to_csv('video_features.csv')

In [50]:
print(len(video_features))
print(len(labels))

92
92


In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

data = pad_sequences(data, padding='post')
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.1, random_state=42, stratify=labels )


In [13]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.1)

ValueError: too many values to unpack (expected 2)

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_df_features = pad_sequences(train_df['features'], maxlen=72, padding='post')
train_df_labels= train_df['class'].values
train_df_labels_one_hot = to_categorical(train_df_labels, num_classes=4)
test_df_labels= test_df['class'].values
test_df = pad_sequences(test_df['features'], maxlen=72, padding='post')
test_df_labels_one_hot = to_categorical(test_df_labels, num_classes=4)

TypeError: list indices must be integers or slices, not str

## Training the model

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.utils.rnn as rnn_utils

device = torch.device("cpu")
lengths_tensor = torch.tensor(lengths)
# mask = torch.arange(train_df_features.size)<lengths_tensor[:, None]
# masked_x = rnn_utils.pack_sequence([torch.masked_select(xi, mi) for xi, mi in zip(train_df_features, mask)])
train_features = torch.tensor(train_data).float().to(device)
train_labels = torch.tensor(train_labels).long().to(device)

test_features = torch.tensor(test_data).float().to(device)
test_labels = torch.tensor(test_labels).long().to(device)

threshold = 0 

In [18]:


# mask_tensor = (train_features>threshold).float()


class Model(nn.Module):
    def __init__(self, num_classes, num_hidden1, num_hidden2):
        super(Model, self).__init__()
        self.conv1 = nn.Conv3d(72, num_hidden1, kernel_size=(3, 3, 3), stride=1, padding=0)
        # self.pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.pool1 = nn.AdaptiveMaxPool3d((None,112,112))
        self.conv2 = nn.Conv3d(num_hidden1, num_hidden2, kernel_size=(3, 3, 3), stride=1, padding=1)
        # self.pool2 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.pool2 = nn.AdaptiveMaxPool3d((None, 52,52))
        self.flatten = nn.Flatten()
        
        num_features = num_hidden2*52*52
        self.fc1 = nn.Linear(num_features, 128)  # Update the number of input features
        self.fc2 = nn.Linear(128, num_classes)
        self.mask = None 
    
    def apply_mask(self, x, mask): 
        return x * mask
    
    def forward(self, x, mask=None):
        if mask is not None: 
            x= self.apply_mask(x,mask)
        x = x.permute(0, 1, 4, 2, 3)  # Swap the depth and width dimensions
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return nn.functional.softmax(x, dim=1)

model = Model(actions.shape[0],48,64).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


train_dataset = TensorDataset(train_features, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # Decrease the batch size

test_dataset = TensorDataset(test_features, test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [20]:
accumulation_steps = 4
num_epochs = 100

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    for i, (features, labels) in enumerate(train_loader):
        features = features.to(device)
        labels = labels.to(device)
        
        mask_tensor = (features > threshold).float().to(device)
        # mask_tensor = mask_condition.any(dim=1, keepdim=True).any(dim=3, keepdim=True).any(dim=4, keepdim=True)
        outputs = model(features,mask_tensor)
        loss = criterion(outputs, labels)
        loss = loss / accumulation_steps
        loss.backward()

        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        running_loss += loss.item() * accumulation_steps

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / (i + 1)
    epoch_acc = 100 * correct / total
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")


Epoch [1/100], Loss: 1.4484, Accuracy: 24.39%
Epoch [2/100], Loss: 1.4389, Accuracy: 24.39%
Epoch [3/100], Loss: 1.4565, Accuracy: 24.39%
Epoch [4/100], Loss: 1.4326, Accuracy: 24.39%
Epoch [5/100], Loss: 1.4629, Accuracy: 24.39%
Epoch [6/100], Loss: 1.4507, Accuracy: 24.39%
Epoch [7/100], Loss: 1.4433, Accuracy: 24.39%
Epoch [8/100], Loss: 1.4600, Accuracy: 24.39%
Epoch [9/100], Loss: 1.4589, Accuracy: 24.39%
Epoch [10/100], Loss: 1.4337, Accuracy: 24.39%
Epoch [11/100], Loss: 1.4724, Accuracy: 24.39%
Epoch [12/100], Loss: 1.4514, Accuracy: 24.39%
Epoch [13/100], Loss: 1.4631, Accuracy: 24.39%
Epoch [14/100], Loss: 1.4439, Accuracy: 24.39%
Epoch [15/100], Loss: 1.4676, Accuracy: 24.39%
Epoch [16/100], Loss: 1.4669, Accuracy: 24.39%
Epoch [17/100], Loss: 1.4586, Accuracy: 24.39%
Epoch [18/100], Loss: 1.4286, Accuracy: 24.39%
Epoch [19/100], Loss: 1.4573, Accuracy: 24.39%
Epoch [20/100], Loss: 1.4530, Accuracy: 24.39%
Epoch [21/100], Loss: 1.4403, Accuracy: 24.39%
Epoch [22/100], Loss: 

In [25]:
train_features.shape

torch.Size([82, 72, 224, 224, 3])

In [34]:
train_labels.shape

torch.Size([82, 4])

In [24]:
correct = 0
total = 0

# Set the model to evaluation mode
model.eval()

# Calculate accuracy on the training set
with torch.no_grad():
    for features, labels in test_loader:
        features = features.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(features)

        # Get the predicted class labels
        _, predicted = torch.max(outputs, 1)

        # Get the ground truth class labels
        true_labels = labels

        # Update the count of total and correct predictions
        total += true_labels.size(0)
        correct += (predicted == true_labels).sum().item()

# Calculate the accuracy
accuracy = 100 * correct / total

print(f'Training accuracy: {accuracy}%')


Training accuracy: 30.0%


In [25]:
print(predicted)
print(true_labels)

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
tensor([0, 0, 2, 1, 3, 2, 0, 1, 2, 3])


In [40]:
torch.cuda.empty_cache()

In [63]:
print("Active device:", device)

Active device: cpu


## Hyper parameter tuning

In [15]:
accumulation_steps = 4 
num_epochs = 1000
def train_and_evaluate_model(trial, train_loader, test_loader): 
    learning_rate = trial.suggest_categorical("learning_rate", [0.0001, 0.001, 0.01])
    num_hidden1 = trial.suggest_int("num_hidden1",16,64, step=16 )
    num_hidden2 = trial.suggest_int("num_hidden2", 16, 64, step=16)
    
    model = Model(actions.shape[0], num_hidden1, num_hidden2).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr= learning_rate)
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for i, (features, labels) in enumerate(train_loader):
            features = features.to(device)
            labels = labels.to(device)
            
            mask_tensor = (features > threshold).float().to(device)
            # mask_tensor = mask_condition.any(dim=1, keepdim=True).any(dim=3, keepdim=True).any(dim=4, keepdim=True)
            outputs = model(features,mask_tensor)
            loss = criterion(outputs, torch.max(labels, 1)[1])
            loss = loss / accumulation_steps
            loss.backward()

            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

            running_loss += loss.item() * accumulation_steps
             


            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == torch.max(labels, 1)[1]).sum().item()
        epoch_loss = running_loss / (i + 1)
        epoch_acc = 100 * correct / total
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")
    correct_test = 0
    total_test = 0

    # Set the model to evaluation mode
    model.eval()

    # Calculate accuracy on the training set
    with torch.no_grad():
        for features, labels in test_loader:
            features = features.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(features)

            # Get the predicted class labels
            _, predicted = torch.max(outputs, 1)

            # Get the ground truth class labels
            _, true_labels = torch.max(labels, 1)

            # Update the count of total and correct predictions
            total_test += true_labels.size(0)
            correct_test += (predicted == true_labels).sum().item()

    # Calculate the accuracy
    accuracy = 100 * correct_test / total_test

    print(f'Testing accuracy: {accuracy}%')
    
    return accuracy 
    


In [13]:
import optuna

In [16]:
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: train_and_evaluate_model(trial, train_loader, test_loader), n_trials=64)

print("Best trial:")
print("  Params: ", study.best_trial.params)
print("  Validation accuracy: ", study.best_trial.value)

[I 2023-04-28 21:23:06,813] A new study created in memory with name: no-name-7adad7d6-83b0-49d8-990e-71679d1f9ee8


Epoch [1/1000], Loss: 1.4376, Accuracy: 23.17%
Epoch [2/1000], Loss: 1.4160, Accuracy: 23.17%
Epoch [3/1000], Loss: 1.4315, Accuracy: 23.17%
Epoch [4/1000], Loss: 1.4235, Accuracy: 23.17%
Epoch [5/1000], Loss: 1.4263, Accuracy: 23.17%
Epoch [6/1000], Loss: 1.4188, Accuracy: 23.17%
Epoch [7/1000], Loss: 1.4138, Accuracy: 23.17%
Epoch [8/1000], Loss: 1.4218, Accuracy: 23.17%
Epoch [9/1000], Loss: 1.4092, Accuracy: 23.17%
Epoch [10/1000], Loss: 1.4149, Accuracy: 23.17%
Epoch [11/1000], Loss: 1.4261, Accuracy: 23.17%
Epoch [12/1000], Loss: 1.3905, Accuracy: 23.17%
Epoch [13/1000], Loss: 1.4163, Accuracy: 23.17%
Epoch [14/1000], Loss: 1.4229, Accuracy: 23.17%
Epoch [15/1000], Loss: 1.4224, Accuracy: 23.17%
Epoch [16/1000], Loss: 1.4158, Accuracy: 23.17%
Epoch [17/1000], Loss: 1.4422, Accuracy: 23.17%
Epoch [18/1000], Loss: 1.4251, Accuracy: 23.17%
Epoch [19/1000], Loss: 1.4324, Accuracy: 23.17%
Epoch [20/1000], Loss: 1.4120, Accuracy: 23.17%
Epoch [21/1000], Loss: 1.4049, Accuracy: 23.17%
E

[I 2023-04-29 04:22:29,360] Trial 0 finished with value: 20.0 and parameters: {'learning_rate': 0.01, 'num_hidden1': 64, 'num_hidden2': 48}. Best is trial 0 with value: 20.0.


Epoch [1/1000], Loss: 1.4169, Accuracy: 19.51%
Epoch [2/1000], Loss: 1.4020, Accuracy: 19.51%
Epoch [3/1000], Loss: 1.4102, Accuracy: 19.51%
Epoch [4/1000], Loss: 1.4061, Accuracy: 19.51%
Epoch [5/1000], Loss: 1.4116, Accuracy: 19.51%
Epoch [6/1000], Loss: 1.4094, Accuracy: 19.51%
Epoch [7/1000], Loss: 1.4094, Accuracy: 19.51%
Epoch [8/1000], Loss: 1.4170, Accuracy: 19.51%
Epoch [9/1000], Loss: 1.4091, Accuracy: 19.51%
Epoch [10/1000], Loss: 1.4083, Accuracy: 19.51%
Epoch [11/1000], Loss: 1.4081, Accuracy: 19.51%
Epoch [12/1000], Loss: 1.4124, Accuracy: 19.51%
Epoch [13/1000], Loss: 1.4055, Accuracy: 19.51%
Epoch [14/1000], Loss: 1.4131, Accuracy: 19.51%
Epoch [15/1000], Loss: 1.4154, Accuracy: 19.51%
Epoch [16/1000], Loss: 1.4056, Accuracy: 19.51%
Epoch [17/1000], Loss: 1.4172, Accuracy: 19.51%
Epoch [18/1000], Loss: 1.4001, Accuracy: 19.51%
Epoch [19/1000], Loss: 1.4002, Accuracy: 19.51%
Epoch [20/1000], Loss: 1.4097, Accuracy: 19.51%
Epoch [21/1000], Loss: 1.4063, Accuracy: 19.51%
E

[I 2023-04-29 10:26:14,230] Trial 1 finished with value: 40.0 and parameters: {'learning_rate': 0.0001, 'num_hidden1': 48, 'num_hidden2': 64}. Best is trial 1 with value: 40.0.


...

Testing accuracy: 20.0%
Epoch [1/1000], Loss: 1.4134, Accuracy: 24.39%
Epoch [2/1000], Loss: 1.4108, Accuracy: 24.39%
Epoch [3/1000], Loss: 1.4119, Accuracy: 24.39%
Epoch [4/1000], Loss: 1.4085, Accuracy: 24.39%
Epoch [5/1000], Loss: 1.4120, Accuracy: 24.39%
Epoch [6/1000], Loss: 1.4149, Accuracy: 24.39%
Epoch [7/1000], Loss: 1.4104, Accuracy: 24.39%
Epoch [8/1000], Loss: 1.4222, Accuracy: 24.39%
Epoch [9/1000], Loss: 1.4014, Accuracy: 24.39%
Epoch [10/1000], Loss: 1.4032, Accuracy: 24.39%
Epoch [11/1000], Loss: 1.4144, Accuracy: 24.39%
Epoch [12/1000], Loss: 1.4131, Accuracy: 24.39%
Epoch [13/1000], Loss: 1.4194, Accuracy: 24.39%
Epoch [14/1000], Loss: 1.4156, Accuracy: 24.39%
Epoch [15/1000], Loss: 1.4125, Accuracy: 24.39%
Epoch [16/1000], Loss: 1.4113, Accuracy: 24.39%
Epoch [17/1000], Loss: 1.4091, Accuracy: 24.39%
Epoch [18/1000], Loss: 1.4202, Accuracy: 24.39%
Epoch [19/1000], Loss: 1.4111, Accuracy: 24.39%
Epoch [20/1000], Loss: 1.4085, Accuracy: 24.39%
Epoch [21/1000], Loss: 1.

[I 2023-05-02 07:08:18,323] Trial 15 finished with value: 20.0 and parameters: {'learning_rate': 0.0001, 'num_hidden1': 64, 'num_hidden2': 48}. Best is trial 1 with value: 40.0.


Testing accuracy: 20.0%
Epoch [1/1000], Loss: 1.4355, Accuracy: 19.51%
Epoch [2/1000], Loss: 1.4347, Accuracy: 19.51%
Epoch [3/1000], Loss: 1.4447, Accuracy: 19.51%
Epoch [4/1000], Loss: 1.4303, Accuracy: 19.51%
Epoch [5/1000], Loss: 1.4297, Accuracy: 19.51%
Epoch [6/1000], Loss: 1.4452, Accuracy: 19.51%
Epoch [7/1000], Loss: 1.4259, Accuracy: 19.51%
Epoch [8/1000], Loss: 1.4356, Accuracy: 19.51%
Epoch [9/1000], Loss: 1.4340, Accuracy: 19.51%
Epoch [10/1000], Loss: 1.4293, Accuracy: 19.51%
Epoch [11/1000], Loss: 1.4408, Accuracy: 19.51%
Epoch [12/1000], Loss: 1.4302, Accuracy: 19.51%
Epoch [13/1000], Loss: 1.4282, Accuracy: 19.51%
Epoch [14/1000], Loss: 1.4419, Accuracy: 19.51%
Epoch [15/1000], Loss: 1.4413, Accuracy: 19.51%
Epoch [16/1000], Loss: 1.4396, Accuracy: 19.51%
Epoch [17/1000], Loss: 1.4316, Accuracy: 19.51%
Epoch [18/1000], Loss: 1.4349, Accuracy: 19.51%
Epoch [19/1000], Loss: 1.4270, Accuracy: 19.51%
Epoch [20/1000], Loss: 1.4381, Accuracy: 19.51%
Epoch [21/1000], Loss: 1.

[W 2023-05-02 10:06:17,871] Trial 16 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "c:\Users\amyan\anaconda3\envs\IPCV\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-16-dcd4089b8b26>", line 2, in <lambda>
    study.optimize(lambda trial: train_and_evaluate_model(trial, train_loader, test_loader), n_trials=64)
  File "<ipython-input-15-704fa1227e83>", line 21, in train_and_evaluate_model
    outputs = model(features,mask_tensor)
  File "c:\Users\amyan\anaconda3\envs\IPCV\lib\site-packages\torch\nn\modules\module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "<ipython-input-11-651615ce7d70>", line 25, in forward
    x= self.apply_mask(x,mask)
  File "<ipython-input-11-651615ce7d70>", line 21, in apply_mask
    return x * mask
KeyboardInterrupt


KeyboardInterrupt: 